<p align="center">Міністерство освіти і науки України НТУУ </p>
<p align="center">«Київський політехнічний інститут імені Ігоря Сікорського»</p>
<p align="center">Фізико-технічний інститут</p>


<p align="center"><strong>Спеціальні розділи програмування</strong></p>

<p align="center">Лаболаторна робота №2</p>

<p align="right">Виконав:</p>
<p align="right">Студент 2 курсу ФТІ</p>
<p align="right">Групи ФІ-31</p>

<p align="right">Горбач Ангеліна</strong></p>

# Item 2

In [66]:
import urllib.request
from datetime import datetime

def download_vhi_data(province_id):
    url = f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1=1981&year2=2024&type=Mean'
    
    now = datetime.now()
    date_and_time = now.strftime("%d%m%Y%H%M%S")
    
    response = urllib.request.urlopen(url)
    data = response.read()

    filename = f'vhi_ID_{province_id}_{date_and_time}.csv'
    with open(filename, 'wb') as file:
        file.write(data)

    print(f"Дані для області {province_id} збережені у файл: {filename}")

for province_id in range(1, 26):
     download_vhi_data(province_id)


# Item 3

In [67]:
import pandas as pd
import os
def load_vhi_data_from_directory(directory_path):
    columns = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    
    all_data = []

    for filename in os.listdir(directory_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory_path, filename)
            
            df = pd.read_csv(file_path, header=1, names=columns)
            df = df.drop(df[df['VHI'] == -1].index)
        
            province_id = filename.split('_')[2]
            df['Province_ID'] = province_id
                  
        

            df=df.drop(columns=['empty'])
            df['Year']=df['Year'].str.replace('<tt><pre>', '')

            df=df.dropna()

            all_data.append(df)

        
    
    combined_data = pd.concat(all_data, ignore_index=True)
    
    return combined_data

directory = r'C:\Users\angel\OneDrive\Робочий стіл\123\lab_2'
vhi_data = load_vhi_data_from_directory(directory)


print(vhi_data.tail())


        Year  Week    SMN     SMT    VCI   TCI    VHI Province_ID
163045  2024  36.0  0.179  303.63  26.33  9.65  18.00           9
163046  2024  37.0  0.173  302.32  26.98  7.14  17.08           9
163047  2024  38.0  0.168  301.00  28.54  4.92  16.74           9
163048  2024  39.0  0.163  299.44  31.24  2.78  17.02           9
163049  2024  40.0  0.160  297.71  34.48  1.43  17.96           9


# Task 4

In [68]:

province_mapping = {
    '1': 'Вінницька', '2': 'Волинська', '3': 'Дніпропетровська', 
    '4': 'Донецька', '5': 'Житомирська', '6': 'Закарпатська', 
    '7': 'Запорізька', '8': 'Івано-Франківська', '9': 'Київська', 
    '10': 'Кіровоградська', '11': 'Луганська', '12': 'Львівська', 
    '13': 'Миколаївська', '14': 'Одеська', '15': 'Полтавська', 
    '16': 'Рівенська', '17': 'Сумська', '18': 'Тернопільська', 
    '19': 'Харківська', '20': 'Херсонська', '21': 'Хмельницька', 
    '22': 'Черкаська', '23': 'Чернівецька', '24': 'Чернігівська', 
    '25': 'Республіка Крим'
}

def update_province_indices(df):
    df['Province_Name'] = df['Province_ID'].replace(province_mapping)
    return df

vhi_data = update_province_indices(vhi_data)
print("Оновлені індекси областей:\n", vhi_data[['Province_ID', 'Province_Name']].drop_duplicates())

Оновлені індекси областей:
        Province_ID      Province_Name
0               10     Кіровоградська
6522            11          Луганська
13044           12          Львівська
19566           13       Миколаївська
26088           14            Одеська
32610           15         Полтавська
39132           16          Рівенська
45654           17            Сумська
52176           18      Тернопільська
58698           19         Харківська
65220            1          Вінницька
71742           20         Херсонська
78264           21        Хмельницька
84786           22          Черкаська
91308           23        Чернівецька
97830           24       Чернігівська
104352          25    Республіка Крим
110874           2          Волинська
117396           3   Дніпропетровська
123918           4           Донецька
130440           5        Житомирська
136962           6       Закарпатська
143484           7         Запорізька
150006           8  Івано-Франківська
156528           9    

## Task 5.1


Ряд VHI для області за вказаний рік;

In [69]:
def vhi_region(index_region, year):
    result = vhi_data[(vhi_data['Province_ID'] == index_region) & (vhi_data['Year'] == year)][['Year', 'VHI', 'Province_ID']]
    return result


In [70]:
print(vhi_region("14", "2005"))

       Year    VHI Province_ID
27235  2005  72.33          14
27236  2005  73.85          14
27237  2005  73.49          14
27238  2005  72.25          14
27239  2005  69.86          14
...     ...    ...         ...
31629  2005  36.83          14
31630  2005  38.98          14
31631  2005  39.76          14
31632  2005  41.39          14
31633  2005  43.90          14

[153 rows x 3 columns]


## Task 5.2

Пошук екстремумів (min та max) для вказаних областей та років,
середнього, медіани;

In [71]:
def extreme(year, regions):
    vhi_data['Year'] = pd.to_numeric(vhi_data['Year'], errors='coerce')
    vhi_data['Province_ID'] = pd.to_numeric(vhi_data['Province_ID'], errors='coerce')
    data = vhi_data[(vhi_data['Year'] >= year[0]) & 
                    (vhi_data['Year'] <= year[1]) & 
                    (vhi_data['Province_ID'] >= regions[0]) & 
                    (vhi_data['Province_ID'] <= regions[1])]

    min_v = data['VHI'].min()
    max_v = data['VHI'].max()
    mean_v = data['VHI'].mean()
    median_v = data['VHI'].median()

    return (f"Extremes:\nMin: {min_v}\nMax: {max_v}\nMean: {mean_v}\nMedian: {median_v}")


print(extreme([1990, 2000], [5, 10]))


Extremes:
Min: 6.26
Max: 96.18
Mean: 47.14389692585895
Median: 45.504999999999995


## Task 5.3

Ряд VHI за вказаний діапазон років для вказаних областей;

In [72]:
def range(year, region):
    
    data = vhi_data[(vhi_data['Year'] >= year[0]) & (vhi_data['Year'] <= year[1]) & 
                    (vhi_data['Province_ID'] >= region[0]) & (vhi_data['Province_ID'] <= region[1])]
    
    filtered_data = data.sort_values(by="Province_ID")
    return filtered_data [['Year','VHI','Province_ID']]

print(range([2000, 2004], [4, 12]))

        Year    VHI  Province_ID
124827  2000  36.23            4
124826  2000  35.84            4
124825  2000  35.78            4
129385  2004  51.14            4
129384  2004  50.47            4
...      ...    ...          ...
18508   2004  70.56           12
18507   2004  74.02           12
13952   2000  25.44           12
13951   2000  23.11           12
13950   2000  18.21           12

[6507 rows x 3 columns]


## Task 5.4

Для всього набору даних виявити роки, протягом яких екстремальні
посухи торкнулися більше вказаного відсотка областей по Україні (20%
областей - 5 областей з 25). Повернути роки, назви областей з
екстремальними посухами та значення VHI;

In [73]:
def droughts(vhi_data, percent):

    count = int((percent * 25) / 100)
    drought = vhi_data[(vhi_data['VHI'] < 15)&(vhi_data['VHI']!=-1)]
    number_of_regions = drought.groupby(by='Year')['Province_ID'].count()
    years_d = number_of_regions[number_of_regions > count].index
    
    result = drought[drought['Year'].isin(years_d)][['Year', 'Province_ID', 'VHI']]
    return result



In [74]:
print(droughts(vhi_data, 20))

        Year  Province_ID    VHI
7471    2000           11  12.51
7472    2000           11  10.60
7473    2000           11  11.20
7474    2000           11  12.32
7475    2000           11  14.65
...      ...          ...    ...
162154  2007            9  12.96
162155  2007            9  13.48
162156  2007            9  14.05
162157  2007            9  14.41
162158  2007            9  14.81

[309 rows x 3 columns]
